### Informações e estatísticas da base de redações Essays Text Dataset (ETD)

In [ ]:
from pathlib import Path
from glob import glob

PATH_REDACOES = "./redacoes/TXT/"

In [ ]:
# Conta da quantidade de arquivos de redações

path = Path(PATH_REDACOES)

all_arqs = [str(x) for x in list(path.rglob("*"))]
arqs_essays = sorted(list(filter(lambda x: ".html" != str(x)[-5:], all_arqs)), 
                     key=lambda x: int(x.split('_')[1]))
arqs_annotated = list(filter(lambda x: ".html" == str(x)[-5:], all_arqs))
#arqs_essays = list(path.glob("*"))
#arqs_annotated = list(path.rglob('*.html'))

print(f"Total de arquivos de redações: {len(arqs_essays)}")        
print(f"Total de arquivos de redações anotadas: {len(arqs_annotated)}")        

In [ ]:
import spacy
nlp = spacy.load('pt_core_news_lg')


In [ ]:
# Contagem de palavras das redacoes
total_palavras = 0
for arq_essay in arqs_essays:
    texto = ""
    with open(arq_essay, "r") as f_essay:
        texto = "\n".join(f_essay.readlines())

    doc = nlp(texto)
    #words = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    words = [token.text for token in doc if token.is_punct != True]
    total_palavras += len(words)

In [ ]:
print(f"Total de palavras de todas as redações: {total_palavras}")
print(f"Média de palavras por redação: {round(total_palavras/len(arqs_essays))}")

In [ ]:
doc_ex = nlp(arqs_essays[0])

Algumas redações estavam com pontuações sem espaços subsequentes ou parênteses antecedentes,
bem como algumas aspas e aspas duplas estavam desnormalizadas

In [ ]:
path_redacao = arqs_essays[0]

texto_redacao = ""
with open(path_redacao) as f:
    texto_redacao = "".join(f.readlines())

texto_redacao 

In [ ]:
import re

# Corrige os textos dos arquivos, alterando
# principalmente, situações de pontuação 
# sem espaço subsequente
def ajusta_texto(texto):
    
    pats = [r"\..[A-Za-z^\s]",   # ponto "." sem espaço subsequente
            r",.[A-Za-z^\s]",    # virgula "," sem espaço subsequente
            r"\(.[A-Za-z^\s]",    # parentese "(" sem espaço antecedente
            r".[A-Za-z^\s]\)"    # parentese "(" sem espaço subsequente
           ]
    chars = [".", ",", "(", ")"]
     
    for c, pat in zip(chars, pats):
        matches = re.findall(pat, texto, re.MULTILINE)         

        for m in matches:
            pos_m = texto.find(m)
            pos_dot = m.find(c)
            
            if c == "(":
                if not(texto[pos_m-1] == " "):
                    m_rep =  f" {m}"
                    texto = texto.replace(m, m_rep)
            else:
                pos_m = pos_m + pos_dot
                if ((pos_m+1) < len(texto)):
                    if not(texto[pos_m+1] == " "):
                        if c == ',' and texto[pos_m+1] in "0123456789":
                            continue
                        m_rep = f"{m[:pos_dot+1]} {m[pos_dot+1:]}"
                        texto = texto.replace(m, m_rep)
    
    texto = texto.replace("\'\'", "\"")
    texto = texto.replace(" .", ".")
    texto = texto.replace(" ,", ",")
    
    return texto

In [ ]:
print(texto_redacao)
print(100*"=")
print(ajusta_texto(texto_redacao))

In [ ]:
s1 = "1,23 milhões ver sem amigos , pois faz. parte(ou mais parte),ou não faria e,outra vez(quem sabe) . "
ajusta_texto(s1)

In [ ]:
# Gerando um arquivo contendo todas as redações
PATH_ALL = "./redacoes/"
l_separador = "\n"

for arq_essay in arqs_essays:
    texto = ""
    texto += f"<ARQUIVO: \"{arq_essay}\">"
    texto += l_separador    
    with open(arq_essay, "r") as f_essay:
        texto += ajusta_texto("".join(f_essay.readlines()))
    
    texto += l_separador        
    with open(f"{PATH_ALL}all_texts.txt", "a") as f_todos:
        f_todos.write(texto)
       
    
    

### Avaliando os erros ortograficos presentes nos arquivos

In [ ]:
import language_tool_python as ltp

tool = ltp.LanguageTool('pt-BR')


In [ ]:
path_redacao = arqs_essays[0]

texto_redacao = ""
with open(path_redacao) as f:
    texto_redacao = "".join(f.readlines())

texto_redacao 
erros = tool.check(ajusta_texto(texto_redacao))

erros